In [218]:
import rasterio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import fiona
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import Polygon
from scipy.ndimage.filters import median_filter, gaussian_filter, gaussian_filter1d
%matplotlib

Using matplotlib backend: Qt5Agg


In [104]:
dem =rasterio.open('/home/cparr/surfaces/level_1_surfaces/hv/bare_earth/hv_dem_final.tif')
dem.meta

{'count': 1,
 'crs': CRS({'init': 'epsg:32606'}),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 11360,
 'nodata': nan,
 'transform': Affine(1.0, 0.0, 421845.75,
       0.0, -1.0, 7673711.25),
 'width': 2340}

In [17]:
src17 = rasterio.open('/home/cparr/surfaces/depth_ddems/hv_version2/hv_depth_apr12_2017.tif')
src17.meta

{'count': 1,
 'crs': CRS({'init': 'epsg:32606'}),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 11360,
 'nodata': nan,
 'transform': Affine(1.0, 0.0, 421845.75,
       0.0, -1.0, 7673711.25),
 'width': 2340}

In [18]:
src16 = rasterio.open('/home/cparr/surfaces/depth_ddems/hv_version2/hv_depth_096_2016.tif')
src16.meta

{'count': 1,
 'crs': CRS({'init': 'epsg:32606'}),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 11360,
 'nodata': nan,
 'transform': Affine(1.0, 0.0, 421845.8,
       0.0, -1.0, 7673711.2),
 'width': 2340}

In [19]:
src15 = rasterio.open('/home/cparr/surfaces/depth_ddems/hv_version2/hv_depth_096_2015.tif')
src15.meta

{'count': 1,
 'crs': CRS({'init': 'epsg:32606'}),
 'driver': 'GTiff',
 'dtype': 'float32',
 'height': 11360,
 'nodata': nan,
 'transform': Affine(1.0, 0.0, 421845.8,
       0.0, -1.0, 7673711.2),
 'width': 2340}

In [113]:
show(src17, cmap ='coolwarm',vmin=0,vmax=2)

In [133]:
def crop_and_test(src, center, size, title):
    
    '''
    This function is responsible for creating a subset of the full dataset and then testing
    different thresholds by masking out data at that threshold.
    The function returns a few plots and a DataFrame which stores the results.
    '''
    
    # Generate a square polygon to crop the full domain
    center_x = center[0]
    center_y = center[1]
    
    poly = Polygon([(center_x-(size / 2), center_y-(size / 2)),
           (center_x - (size / 2), center_y + (size / 2)),
           (center_x + (size / 2), center_y + (size / 2)),
           (center_x + (size / 2), center_y - (size / 2)),
           (center_x - (size / 2), center_y - (size / 2))])
    
    geom = [p for p in poly.boundary.coords]
    cropper = [{'coordinates': [geom],
              'type': 'Polygon'}]
    
    # Crop it and return the cropped data and affine transform
    out_image, out_transform = mask(src, cropper, crop=True)
    out_meta = src.meta.copy()
    
    # Calculate some stats
    mu = np.nanmean(out_image.data[0])
    sigma = np.nanstd(out_image.data[0])
    dmin = np.nanmin(out_image.data[0])
    dmax = np.nanmax(out_image.data[0])
    
    surface = out_image.data[0]
    surface = np.nan_to_num(surface)

    
    # Generate Figures
    
    plt.figure(figsize = (16,10))
    ax=plt.subplot(111)
    
    plt.imshow(median_filter(out_image.data[0], 2), cmap='coolwarm', vmin = 0, vmax = 1)
    
    ax.set_ylabel('m')
    ax.set_xlabel('m')
    plt.title(title +'\n'+ 
              "$\mu=%.2f$, $\sigma=%.2f$"  %(mu,sigma))
    plt.colorbar()
    
    return surface, mu, sigma

In [204]:
#polygons = (422800, 7669000)
polygons = (423260, 7664500)
poly17 = crop_and_test(center=polygons, size=1000, src=src17, title='Polygons 2017 [m]')

In [205]:
poly16 = crop_and_test(center=polygons, size=1000, src=src16, title='Polygons 2016 [m]')

In [206]:
poly15 = crop_and_test(center=polygons, size=1000, src=src15, title='Polygons 2015 [m]')

In [207]:
hv_dem = crop_and_test(center=polygons, size=1000, src=dem, title='Polygons 2015 [m]')[0]

In [202]:
fig = plt.figure(figsize = (8,5))
fig.subplots_adjust(wspace=0.05, hspace=0.05)

ax1 = fig.add_subplot(131)
im1 = ax1.imshow(median_filter(poly15[0],3), cmap='coolwarm', vmin=0,vmax=1.5)
ax1.set_ylabel('m')
ax1.set_xlabel('m')
ax1.set_xticks([0,200,400])
mu = poly15[1]
sigma = poly15[2]
title = '2015'
ax1.set_title(title +'\n'+ 
          "$\mu=%.2f$, $\sigma=%.2f$"  %(mu,sigma))

ax2 = fig.add_subplot(132)
ax2.set_xticks([])
ax2.set_yticks([])
im2 = ax2.imshow(median_filter(poly16[0],3), cmap='coolwarm', vmin=0,vmax=1.5)
mu = poly16[1]
sigma = poly16[2]
title = '2016'
ax2.set_title(title +'\n'+ 
          "$\mu=%.2f$, $\sigma=%.2f$"  %(mu,sigma))

ax3 = fig.add_subplot(133)
ax3.set_xticks([])
ax3.set_yticks([])
im3 = ax3.imshow(median_filter(poly17[0],3), cmap='coolwarm', vmin=0,vmax=1.5)
mu = poly17[1]
sigma = poly17[2]
title = '2017'
ax3.set_title(title +'\n'+ 
          "$\mu=%.2f$, $\sigma=%.2f$"  %(mu,sigma))

cax = fig.add_axes( [0.25, 0.05, 0.5, 0.055] )
fig.colorbar( im1, cax=cax, ticks = ([0,0.5,1.0,1.5,1]), orientation='horizontal' )
cax.tick_params(labelsize = 12)

#fig.savefig('/home/cparr/Snow_Patterns/figures/hv_proposal_figure_wt.png',bbox_inches='tight',dpi=300) #

In [221]:
dem_line = gaussian_filter(hv_dem,0.5)[700][20:215]
line17 = gaussian_filter(poly17[0],0.5)[700][20:215]
line16 = gaussian_filter(poly16[0],0.5)[700][20:215]
line15 = gaussian_filter(poly15[0],0.5)[700][20:215]



fig = plt.figure(figsize=(11,5))
fig.subplots_adjust(wspace=0.3)
ax1 = fig.add_subplot(121)

ax1.plot(dem_line - 0.1, label='Ground',color='black')
ax1.plot(dem_line+line17, label='2017 Snow')
ax1.plot(dem_line+line16, label='2016 Snow')
ax1.plot(dem_line+line15, label='2015 Snow')
ax1.set_xlabel('Distance [m]')
ax1.set_ylabel('Elevation [m]')
ax1.set_xlim(0,195)
ax1.set_ylim(377,395)

x=np.arange(0,195,1)
ax1.fill_between(x, 375, dem_line-0.1, alpha=0.5, color='sienna')
ax1.legend(fontsize=12)

ax2 = fig.add_subplot(122)
ax2.plot(gaussian_filter1d(line17,1), label='2017 Snow')
ax2.plot(gaussian_filter1d(line16,1), label='2016 Snow')
ax2.plot(gaussian_filter1d(line15,1), label='2015 Snow')
ax2.set_xlabel('Distance [m]')
ax2.set_ylabel('Snow Depth [m]')
ax2.set_xlim(0,195)
ax2.set_ylim(0,1.5)
ax2.set_yticks([0,0.25,0.5,0.75,1,1.25,1.5])
ax2.legend(fontsize=12)


fig.savefig('/home/cparr/Snow_Patterns/figures/hv_lake_drift_profiles.png',dpi=300)